In [24]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import json
from qdrant_client.models import SearchRequest
with open('config.json', 'r') as f:
    config = json.load(f)

data_path = config['data_path']

corpus_path=data_path+'corpus.csv'
print(corpus_path)

G:\20232\intern ai cmc\archive\hackathon data backup\corpus.csv


In [3]:
model_name = data_path+"e5_legal_1\run 4\e5_legal_1\checkpoint-21688"
embeddings = SentenceTransformer(r'G:\20232\intern ai cmc\archive\hackathon data backup\e5_legal_1\run 4\e5_legal_1\checkpoint-21688')

You try to use a model that was created with version 3.3.0, however, your version is 3.2.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [4]:
client = QdrantClient(
    path=data_path+'qdrant_db'
)
collection_name = "legal_hackthon_2024_ver3"
print(client.get_collections())
number_points = client.count(collection_name).count
print(number_points)


collections=[CollectionDescription(name='legal_hackthon_2024_ver1'), CollectionDescription(name='legal_hackthon_2024_ver2'), CollectionDescription(name='legal_hackthon_2024_ver3')]
398220


In [41]:
with open(data_path+'finetune results\\run 5\\query_embeddings.json', 'r') as file:
    query_embeddings = json.load(file)

def search_queries_batch(queries, index):
    if type(queries[0]) is str:
        queries = embeddings.encode(queries)
    
    requests=[SearchRequest(vector=vector, limit=30, with_payload=True) for vector in queries]

    answers = client.search_batch(collection_name, requests)
    final_answers=[]
    for idx,i in enumerate(answers):
        curr=[index[idx]]
        curr.extend([point.payload['cid'] for point in i])
        final_answers.append(list(set(curr))[:11])
    return final_answers    

In [36]:
import pandas as pd
df=pd.read_csv(data_path+'public_test.csv')
query_index=df['qid'].to_list()
print(len(query_index))

10000


In [43]:
result=[]
for i in range(0,len(query_embeddings), 500):
    curr_search=search_queries_batch(query_embeddings[i:i+500], query_index[i:i+500])
    result.extend(curr_search)
    

In [53]:
len(query_embeddings)

10000

In [49]:
with open('predict.txt','w') as file:
    for i in result:
        file.write(' '.join([str(x) for x in i]) +'\n')